In [2]:
import hanlp
hanlp_model = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)


100% 114.3 MiB 418.4 KiB/s ETA:  0 s [=============================================================]
Decompressing C:\Users\USER\AppData\Roaming\hanlp\mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip to C:\Users\USER\AppData\Roaming\hanlp\mtl
C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\USER\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pyt

In [6]:
import pandas as pd

# 讀取前 100 筆資料
english_df = pd.read_csv("English_Test_ner_tagged_100.csv").head(100)
chinese_df = pd.read_csv("Chience_Test_ner_tagged_100.csv").head(100)


In [7]:
entity_dict = {}

# 遍歷中英文資料集的每一筆數據
for index in range(len(english_df)):
    english_entities = eval(english_df.loc[index, 'English NER Tagged'])
    chinese_entities = eval(chinese_df.loc[index, 'Chinese NER Tagged'])
    
    # 將實體對應加入到翻譯詞典中
    for (en_entity, en_tag) in english_entities:
        for (zh_entity, zh_tag, _, _) in chinese_entities:
            # 確保實體類別一致，例如都是人名或地名
            if en_tag == 'PERSON' and zh_tag == 'nr':
                entity_dict[en_entity] = zh_entity
            elif en_tag == 'LOCATION' and zh_tag == 'ns':
                entity_dict[en_entity] = zh_entity
print(entity_dict)


{'Chris': '克里斯 安德森', 'White': '纳什维尔', 'House': '纳什维尔', 'Nashville': '纳什维尔', 'Lebanon': '黎巴嫩市', 'Tennessee': '黎巴嫩市', 'Al': '杰伊莱诺', 'Gore': '杰伊莱诺', 'Tipper': '杰伊莱诺', 'Africa': '拉各斯', 'Nigeria': '美国', 'Lagos': '拉各斯', 'Azores': '亚速尔群岛', 'Islands': '亚速尔群岛', 'Washington': '华盛顿', 'Atlantic': '华盛顿', 'United': '美国', 'States': '美国', 'of': '美国', 'America': '美国', 'Monterey': '美国', 'Shoney': '杰伊莱诺', 'U.S.': '美国', 'David': '杰伊莱诺', 'Letterman': '杰伊莱诺', 'Jay': '杰伊莱诺', 'Leno': '杰伊莱诺', 'Bill': '艾尔', 'Clinton': '艾尔', 'Anderson': '克里斯 安德森', 'Vinod': '约翰·多尔', 'John': '约翰·多尔', 'Doerr': '约翰·多尔', 'Larry': '拉里·莱斯格', 'Lessig': '拉里·莱斯格'}


In [3]:
from fuzzywuzzy import fuzz

# 假設已知的英文和中文實體標註結果
english_entities = [("Chris", "PERSON"), ("Nashville", "LOCATION")]
chinese_entities = [("克里斯", "nr"), ("纳什维尔", "ns")]

# 建立標籤對應表
tag_mapping = {"PERSON": "nr", "LOCATION": "ns", "ORGANIZATION": "nt"}

# 構建翻譯詞典
entity_dict = {}

for en_entity, en_tag in english_entities:
    for zh_entity, zh_tag in chinese_entities:
        # 確認標籤是否對應
        if tag_mapping.get(en_tag) == zh_tag:
            # 檢查名稱相似度
            if fuzz.ratio(en_entity, zh_entity) > 80:  # 相似度閾值
                entity_dict[en_entity] = zh_entity

print("實體翻譯詞典:", entity_dict)


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
from fuzzywuzzy import fuzz

def is_similar(entity_en, entity_zh):
    return fuzz.ratio(entity_en, entity_zh) > 80  # 80 為相似度閾值，可根據需求調整


In [ ]:
# 將標籤統一
def normalize_tag(tag):
    if tag in ["nr", "PERSON"]:
        return "PERSON"
    elif tag in ["ns", "LOCATION"]:
        return "LOCATION"
    elif tag in ["nt", "ORGANIZATION"]:
        return "ORGANIZATION"
    return tag

# 將英中文實體結果添加到詞典中
for en_entity, en_tag in english_ner_results:
    en_tag = normalize_tag(en_tag)
    for zh_entity, zh_tag in chinese_ner_results:
        zh_tag = normalize_tag(zh_tag)
        if en_tag == zh_tag:  # 如果標籤相同，認為是對應實體
            entity_dict[en_entity] = {"zh": zh_entity, "tag": en_tag}
            break

print("實體翻譯詞典:", entity_dict)
# 輸出結果應包含 {"Chris": {"zh": "克里斯", "tag": "PERSON"}, "Nashville": {"zh": "纳什维尔", "tag": "LOCATION"}}
